In [1]:
from util_functions import *
import torch

In [7]:
mean_tensor =  torch.randn(10, 2)
log_sigma_tensor = torch.randn(10, 2)**2
monte_carlo_samples = 5

sample_from_multivariantgaussian(mean_tensor, log_sigma_tensor, monte_carlo_samples)[1].shape

torch.Size([10, 2])